In [1]:
import langchain
from langchain_community.document_loaders import PyPDFLoader


In [2]:

def load_pdf(file_path):
    """Loads a PDF and extracts text from it."""
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages

pdf_path = "../ikigai.pdf"
documents = load_pdf(pdf_path)


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_text(documents, chunk_size=500, chunk_overlap=100):
    """Splits extracted text into manageable chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

text_chunks = chunk_text(documents)

for i, chunk in enumerate(text_chunks[:5]):
    print(f"Chunk {i+1}:\n{chunk.page_content}\n")


Chunk 1:
IKIGAI 
1. What is your reason for being? 
2. Whatever you do, don’t retire! 
3. Having a clearly defined ikigai brings satisfaction, 
happiness, and meaning to our lives. 
4. Their blood tests reveal fewer free radicals (which are 
responsible for cellular aging), as a result of drinking tea 
and eating until their stomachs are only 80 percent full 
5. the keys to longevity are diet, exercise, finding a purpose 
in life (an ikigai), and forming strong social ties—that is,

Chunk 2:
in life (an ikigai), and forming strong social ties—that is, 
having a broad circle of friends and good family relations. 
6. Members of these communities manage their time well in 
order to reduce 
7. stress, consume little meat or processed foods, and drink 
alcohol in moderation.1 They don’t do strenuous exercise, 
but they do move every day, taking walks and working in 
their vegetable gardens 
8. Fill your belly to 80 percent.” Ancient wisdom advises

Chunk 3:
their vegetable gardens 
8. Fill 

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import google.generativeai as genai
import faiss
import numpy as np
from langchain_google_genai import GoogleGenerativeAIEmbeddings


# Initialize Gemini Embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


c:\Users\Parth Shethji\Desktop\College\4th Year\sem8\EduRAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def generate_gemini_embeddings(text_chunks):
    """Converts text chunks into Gemini embeddings."""
    texts = [chunk.page_content for chunk in text_chunks]
    embeddings = embeddings_model.embed_documents(texts)
    return np.array(embeddings).astype("float32"), texts

# Convert text chunks to embeddings
embeddings, texts = generate_gemini_embeddings(text_chunks)


In [7]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
faiss.write_index(index, "faiss.index")


In [8]:
# Save text chunks separately
import pickle
with open("stored_chunks.pkl", "wb") as f:
    pickle.dump(texts, f)  # `texts` contains the original text chunks


In [9]:
import faiss
import pickle

# Load FAISS index
index = faiss.read_index("faiss.index")

# Load stored text chunks (stored separately in a pickle file)
with open("stored_chunks.pkl", "rb") as f:
    texts = pickle.load(f)  # This should contain the chunked text


In [10]:
def extract_topics_from_chunks_and_summarize(chunks):
    """Extract key topics from stored FAISS chunks."""
    combined_text = "\n".join(chunks)  # Merge chunks for topic extraction
    model = genai.GenerativeModel('gemini-pro')
    prompt=f"Extract the most important topics from the following study material :\n\n{combined_text}"

    response = model.generate_content(prompt)
    print(response.text.strip())
    

    # topics = response.text.split()  # Convert response to list
    # #topics = topics.replace("'''", "").replace('"""', "").replace("JSON", "").strip()
    prompt=f"Summarize the following and just return the summary:\n\n{combined_text}"
    response = model.generate_content(prompt)
    
    print(response.text.strip())

# Extract topics from the stored chunks
extract_topics_from_chunks_and_summarize(texts)
# important_topics = extract_topics_from_chunks(texts)
# print("\nExtracted Topics:\n", important_topics)


**Important Topics from the Study Material on Ikigai:**

* **Definition of Ikigai:** The reason for being; what brings satisfaction, happiness, and meaning to our lives.
* **Keys to Longevity:** Diet, exercise, finding purpose (ikigai), and strong social ties.
* **The Blue Zone: Okinawa:** People in this region live longer, attributed to diet, exercise, and community support.
* **Hara Hachi Bu:** The practice of eating until 80% full to promote longevity.
* **Moai:** Informal groups in Okinawa that provide emotional and financial support, contributing to life expectancy.
* **Mens Sana in Corpore Sano:** The importance of both mental and physical health for well-being.
* **Stress and Longevity:** Chronic stress can accelerate aging; finding ways to manage stress is crucial.
* **Mindfulness:** Techniques like meditation, breathing exercises, and yoga can help reduce stress and promote well-being.
* **Importance of Staying Active:** Avoid sedentary lifestyles by engaging in regular exerci

In [11]:
def gen_quiz(chunks):
    """Extract key topics from stored FAISS chunks."""
    combined_text = "\n".join(chunks)  # Merge chunks for topic extraction
    model = genai.GenerativeModel('gemini-pro')

    

    # topics = response.text.split()  # Convert response to list
    # #topics = topics.replace("'''", "").replace('"""', "").replace("JSON", "").strip()
    prompt=f"Generate Quiz for the following content: \n\n{combined_text}"
    response = model.generate_content(prompt)
    
    print(response.text.strip())

# Extract topics from the stored chunks
gen_quiz(texts)
# important_topics = extract_topics_from_chunks(texts)
# print("\nExtracted Topics:\n", important_topics)


**Quiz**

**Questions:**

1. What is your reason for being?
2. What is the key to longevity, according to the Okinawan locals?
3. What is the "80 percent rule" for eating?
4. How can meditation help you prevent the negative effects of stress?
5. What is Morita therapy?
6. What are the four phases of Morita therapy?
7. What does it mean to "discover your ikigai"?
8. What are some strategies for achieving flow?
9. How can you cultivate good habits?
10. What is the secret to a long and healthy life, according to the Okinawan elders?

**Answers:**

1. Finding your ikigai, or purpose in life.
2. Diet, exercise, finding a purpose in life, and forming strong social ties.
3. Eating until you are 80 percent full.
4. It helps you reduce stress and calm the mind.
5. A Japanese therapy that emphasizes accepting and observing one's emotions.
6. Isolation and rest, light occupational therapy, occupational therapy, and return to social life.
7. Finding the activities that bring you joy and fulfillmen

In [12]:
def gen_quiz(chunks):

    combined_text = "\n".join(chunks)  # Merge chunks for topic extraction
    model = genai.GenerativeModel('gemini-pro')
    question="what is MOAI?"
    

    # topics = response.text.split()  # Convert response to list
    # #topics = topics.replace("'''", "").replace('"""', "").replace("JSON", "").strip()
    prompt=f"Answer the following question from the give text.\n\ngiven text:{combined_text}\n\n{question}"
    response = model.generate_content(prompt)
    
    print(response.text.strip())

gen_quiz(texts)



Moai is an informal group of people with common interests who look out for one another. In this way, being part of a moai helps maintain emotional and financial stability
